In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_df = pd.read_csv('/kaggle/input/titanic/train.csv')
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')
combine = [train_df, test_df]

In [3]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
train_df.info()
print('_'*40)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Passenger

In [6]:
#Basic stat of the training set
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


# Data cleaning

In [7]:
#As there are some missing value in Column 'Age' and ages are continuated data, we fill the missing data with median
data_clean = pd.concat([train_df, test_df], axis = 0)
data_clean.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [9]:
data_clean['Age'] = data_clean['Age'].fillna(data_clean['Age'].mean())
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1309 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [10]:
data_clean['Embarked'].value_counts()

S    914
C    270
Q    123
Name: Embarked, dtype: int64

In [11]:
data_clean['Embarked'] = data_clean['Embarked'].fillna(data_clean['Embarked'].mode()[0])
data_clean['Fare'] = data_clean['Fare'].fillna(data_clean['Fare'].mean())

In [12]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1309 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1309 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1309 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [13]:
data_clean = data_clean.drop((['Cabin', 'Ticket']), axis = 1)

In [14]:
data_clean['Embarked']

0      S
1      C
2      S
3      S
4      S
      ..
413    S
414    C
415    S
416    S
417    C
Name: Embarked, Length: 1309, dtype: object

In [15]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
label = LabelEncoder()
enc = OneHotEncoder()
data_clean['Embarked'] = data_clean['Embarked'].astype(str)
dum_df = pd.get_dummies(data = data_clean, columns = ['Sex','Embarked'])
dum_df.head()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0.0,3,"Braund, Mr. Owen Harris",22.0,1,0,7.2500,0,1,0,0,1
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,71.2833,1,0,1,0,0
2,3,1.0,3,"Heikkinen, Miss. Laina",26.0,0,0,7.9250,1,0,0,0,1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,53.1000,1,0,0,0,1
4,5,0.0,3,"Allen, Mr. William Henry",35.0,0,0,8.0500,0,1,0,0,1


In [16]:
target = data_clean['Survived']

In [17]:
dum_df = dum_df.drop(['Survived', 'Name'], axis = 1)
dum_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Pclass       1309 non-null   int64  
 2   Age          1309 non-null   float64
 3   SibSp        1309 non-null   int64  
 4   Parch        1309 non-null   int64  
 5   Fare         1309 non-null   float64
 6   Sex_female   1309 non-null   uint8  
 7   Sex_male     1309 non-null   uint8  
 8   Embarked_C   1309 non-null   uint8  
 9   Embarked_Q   1309 non-null   uint8  
 10  Embarked_S   1309 non-null   uint8  
dtypes: float64(2), int64(4), uint8(5)
memory usage: 78.0 KB


# Binning

In [18]:
dum_df['Age'].value_counts(bins = 6, sort = False)

(0.0892, 13.475]     99
(13.475, 26.78]     375
(26.78, 40.085]     608
(40.085, 53.39]     150
(53.39, 66.695]      68
(66.695, 80.0]        9
Name: Age, dtype: int64

In [19]:
# If we choose 6 classes, we would be going with interval of 13
bins = [0, 13, 26, 40, 53, 66, 80]
names = ['<13', '13-26', '26-40', '40-53', '53-66', '66-80']

dum_df['AgeCategory'] = pd.cut(dum_df['Age'], bins, labels = names)

numberedAgeCat = {'<13':0, '13-26':1, '26-40':2, '40-53':3, '53-66':4, '66-80':5}
dum_df['AgeCategory'] = dum_df['AgeCategory'].map(numberedAgeCat)
dum_df['AgeCategory'] = pd.to_numeric(dum_df['AgeCategory'])
dum_df = dum_df.drop(['Age'], axis = 1)

In [20]:
dum_df['Fare'].value_counts(bins = 4, sort = False)
dum_df['FareBin'] = pd.qcut(dum_df['Fare'], 4)
dum_df['FareBin_Code'] = label.fit_transform(dum_df['FareBin'])

In [21]:
dum_df['FamilySize'] = dum_df['Parch'] + dum_df['SibSp'] + 1

dum_df['IsAlone'] = 1
dum_df['IsAlone'].loc[dum_df['FamilySize'] > 1] = 0
dum_df = dum_df.drop(['SibSp', 'Parch', 'FamilySize', 'FareBin'], axis = 1)
dum_df.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,PassengerId,Pclass,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,AgeCategory,FareBin_Code,IsAlone
0,1,3,7.2500,0,1,0,0,1,1,0,0
1,2,1,71.2833,1,0,1,0,0,2,3,0
2,3,3,7.9250,1,0,0,0,1,1,1,1
3,4,1,53.1000,1,0,0,0,1,2,3,0
4,5,3,8.0500,0,1,0,0,1,2,1,1


In [22]:
#Split data to train and test set
train = dum_df.iloc[:891, :]
test = dum_df.iloc[891:, :]
train_y = target.iloc[:891]
X_train = train.drop('PassengerId', axis = 1).copy()
X_test = test.drop('PassengerId', axis = 1).copy()

# Feature scaling

In [23]:
from sklearn.preprocessing import StandardScaler
slr = StandardScaler()
train_std = slr.fit_transform(X_train)
train_std

array([[ 0.82737724, -0.50244517, -0.73769513, ..., -0.82358549,
        -1.34122811, -1.2316449 ],
       [-1.56610693,  0.78684529,  1.35557354, ...,  0.21287591,
         1.36043182, -1.2316449 ],
       [ 0.82737724, -0.48885426,  1.35557354, ..., -0.82358549,
        -0.4406748 ,  0.81192233],
       ...,
       [ 0.82737724, -0.17626324,  1.35557354, ...,  0.21287591,
         0.45987851, -1.2316449 ],
       [-1.56610693, -0.04438104, -0.73769513, ..., -0.82358549,
         0.45987851,  0.81192233],
       [ 0.82737724, -0.49237783, -0.73769513, ...,  0.21287591,
        -1.34122811,  0.81192233]])

# Building predictive model

In [24]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
model = lr.fit(train_std, train_y)
model.score(train_std, train_y)

0.7957351290684624

# Create submission file

In [25]:
test_std = slr.transform(X_test)
y_pred = model.predict(test_std).astype(int)

In [26]:
submission = pd.DataFrame({
    'PassengerId' : test['PassengerId'],
    'Survived' : y_pred
})
submission.to_csv('submission.csv', index=False)